<a href="https://colab.research.google.com/github/wojr21/courses/blob/main/data_science_bootcamp/02_metryki_binarna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### Accuracy - Dokładność

Accuracy = correct predictions / total predictions * 100

In [42]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
                   1, 0 ,1, 1, 0, 1, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
                   1, 0 ,1, 1, 0, 1, 0, 1, 0, 1])
print(y_true, '\n', y_pred)

[1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1] 
 [0 0 1 1 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1]


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.8

In [ ]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,0,0
8,1,1
9,0,1


In [ ]:
results = results.sort_values(by = 'y_true')
results.reset_index(drop = True, inplace = True)
results['sample'] = results.index + 1
results

,y_true,y_pred,sample
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,0,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,1,9
9,0,0,10


In [ ]:
fig = make_subplots(rows = 2, cols = 1)
fig.add_trace(go.Scatter(x = results['sample'], y = results['y_true'],
                         mode = 'markers', name = 'y_true'), row = 1, col = 1)
fig.add_trace(go.Scatter(x = results['sample'], y = results['y_pred'],
                         mode = 'markers', name = 'y_pred'), row = 2, col = 1)
fig.update_layout(width = 900, height = 500, title = 'Klasyfikator binarny')
fig.show()

### Macierz konfuzji

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

In [ ]:
pd.DataFrame(cm, index = ['true_0', 'true_1'],
             columns = ['pred_0', 'pred_1'])

,pred_0,pred_1
true_0,9,1
true_1,5,15


In [ ]:
import plotly.figure_factory as ff

cm_df = pd.DataFrame(cm[::-1], index = ['true_1', 'true_0'],
                     columns = ['pred_0', 'pred_1'])

fig = ff.create_annotated_heatmap(z = cm_df.values, x = list(cm_df.columns),
                                  y = list(cm_df.index), colorscale = 'ice',
                                  showscale = True, reversescale = True)
fig.update_layout(width = 400, height = 400, title = 'Confusion Matrix',
                  font_size = 16)
fig.show()

In [ ]:
tn, fp, fn, tp = cm.ravel()
print(f'True Negative: {tn}')
print(f'False Positive: {fp}')
print(f'False Negative: {fn}')
print(f'True Positive: {tp}')

True Negative: 9
False Positive: 1
False Negative: 5
True Positive: 15


In [ ]:
# Type I Error (False Positive Rate)
fpr = fp / (fp + tn)
fpr

0.1

In [ ]:
# Type II Error (True Negative Rate)
fnr = fn / (fn + tp)
fnr

0.25

In [ ]:
# Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
precision

0.9375

In [ ]:
# Recall - jak wiele obserwacji z wszystkich pozytywnych obserwacji są sklasyfikowane jako pozytywne
recall = tp / (tp + fn)
recall

0.75

In [ ]:
# F1 score - średnia harmoniczna precision i recall
F1_score = 2 / (recall ** -1 + precision ** -1)
F1_score

0.8333333333333334

### Krzywa ROC

In [ ]:
from sklearn.metrics import roc_curve

roc_curve(y_true, y_pred, pos_label = 1)

(array([0. , 0.1, 1. ]), array([0.  , 0.75, 1.  ]), array([2, 1, 0]))

In [ ]:
fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label = 1)
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc

,fpr,tpr
0,0.0,0.00
1,0.1,0.75
2,1.0,1.00


In [ ]:
# Im większe pole między krzywą naszego modelu a linią oznaczającą model losowy, tym lepszy jest ten model

fig = go.Figure(data = [go.Scatter(x = roc['fpr'], y = roc['tpr'],
                                   line_color = 'red', name = 'ROC Curve'),
                        go.Scatter(x = [0, 1], y = [0, 1], mode = 'lines',
                                   line_dash = 'dash', line_color = 'navy')],
                layout = go.Layout(xaxis_title = 'False Positive Rate',
                                   yaxis_title = 'True Positive Rate',
                                   title = 'ROC Curve',
                                   showlegend = False,
                                   width = 600))
fig.show()